In [1]:
import pandas as pd
import numpy as np
import random
import math

data = pd.read_csv('data1.csv')

In [2]:
data['clusterID'] = np.nan

In [3]:
numeric = {'DateVals', 'Number_of_Casualties', 'TimeVals', 'Number_of_Vehicles'}
categor = {'Pedestrian_Crossing-Physical_Facilities', 'Day_of_Week', 'Junction_Detail', 'Road_Surface_Conditions', 'Light_Conditions', 'Did_Police_Officer_Attend_Scene_of_Accident', 'Road_Type', 'Speed_limit', '1st_Road_Class', 'Urban_or_Rural_Area', 'Accident_Severity', 'Weather_Conditions'}
exclude = {'Date', 'Location_Northing_OSGR', 'Latitude', 'Location_Easting_OSGR', 'Time', 'Longitude'}
identif = {'1st_Road_Number',  '2nd_Road_Number', 'Accident_Index', 'Local_Authority_(Highway)', 'Local_Authority_(District)', 'LSOA_of_Accident_Location'}

In [4]:
colTypeDF = pd.DataFrame(columns = ['name', 'type', 'unique'])

colTypeDict = {'Categorical': categor, 'Numerical': numeric, 'Excluded': exclude, 'Identifier': identif}

for k, v in colTypeDict.items():
    for i in v:
        print
        
        vals = {'name': i, 'type': k, 'unique':len(data[i].unique())}
        colTypeDF = colTypeDF.append(vals, ignore_index = True)
    
colTypeDF

,name,type,unique
0,Road_Type,Categorical,4
1,Day_of_Week,Categorical,7
2,Junction_Detail,Categorical,7
3,Urban_or_Rural_Area,Categorical,2
4,1st_Road_Class,Categorical,4
5,Pedestrian_Crossing-Physical_Facilities,Categorical,4
6,Did_Police_Officer_Attend_Scene_of_Accident,Categorical,2
7,Accident_Severity,Categorical,2
8,Weather_Conditions,Categorical,3
9,Speed_limit,Categorical,2


In [5]:
testCols = set.union(numeric, categor)
centrDF = pd.DataFrame(columns = testCols)
centroids = 3

for i in range(centroids):
    vals = {}
    for i in testCols:
        vals[i] = data[i].unique()[random.randint(0, len(data[i].unique()) - 1)]

    centrDF = centrDF.append(vals, ignore_index = True)
    
for i in testCols:
    centrDF[i] = centrDF[i].astype('float')
    
centrDF

,Road_Type,Day_of_Week,Junction_Detail,Light_Conditions,Number_of_Vehicles,1st_Road_Class,Pedestrian_Crossing-Physical_Facilities,Did_Police_Officer_Attend_Scene_of_Accident,Accident_Severity,Road_Surface_Conditions,Speed_limit,DateVals,Urban_or_Rural_Area,Weather_Conditions,Number_of_Casualties,TimeVals
0,3.0,6.0,0.0,4.0,1.0,3.0,0.0,2.0,2.0,2.0,30.0,1998.0,1.0,1.0,1.0,1229.0
1,3.0,2.0,1.0,1.0,1.0,4.0,5.0,2.0,3.0,2.0,40.0,1895.0,2.0,2.0,2.0,602.0
2,1.0,3.0,2.0,4.0,3.0,5.0,4.0,1.0,2.0,2.0,40.0,1860.0,1.0,2.0,2.0,93.0


In [6]:
#checks if the centroids need to be changed
def testCentroids():
    flag = False
    changes = 0
    cols1 = []
    cols2 = []
    
    for i in centrDF.index.tolist():
        df = data.loc[data['clusterID'] == i]
        if(df.shape[0] == 0):
            continue
        for c in testCols:  
            new = df[c].mean()
            cur = centrDF.at[i, c]
        
            if(new != cur):
                flag = True
                centrDF.at[i, c] = new
                changes += 1
                val =c + ': ' + str(new - cur) + ' ' + str(new) + ' ' + str(cur)
                cols1.append(val)

       
    print('changes: ', changes, '\n ', cols1)
    #print('changes: ', changes, '\n ', cols1, '\n', cols2)
    return flag

In [7]:
runs = 0
flag = True

while(flag):
    changes = 0
    
    for i in data.index.tolist():
        distances = []
        vals = data.loc[i].to_dict()
        
        for k in centrDF.index.tolist():
            cVals = centrDF.loc[k].to_dict()
            x = 0
            for d in testCols:
                if(d in numeric):
                    q =  ((cVals[d] - data[d].min()) / (data[d].max() - data[d].min())) - ((vals[d] - data[d].min()) / (data[d].max() - data[d].min())) 
                    x += q**2
                else:
                    if(vals[d] == cVals[d]):
                        x += 1
            distances.append(math.sqrt(x))
        if(data.at[i, 'clusterID'] != distances.index(min(distances))):
            changes += 1
            data.at[i, 'clusterID'] = distances.index(min(distances))
    runs += 1
    
    flag = testCentroids()
    if (changes == 0):
        flag = False
        
    print('runs ', runs, changes)
    
    

changes:  48 
  ['Road_Type: 2.2275747508305646 5.2275747508305646 3.0', 'Day_of_Week: -2.4368770764119603 3.5631229235880397 6.0', 'Junction_Detail: 3.3970099667774085 3.3970099667774085 0.0', 'Light_Conditions: -2.521594684385382 1.478405315614618 4.0', 'Number_of_Vehicles: 0.5963455149501662 1.5963455149501662 1.0', '1st_Road_Class: 1.043189368770764 4.043189368770764 3.0', 'Pedestrian_Crossing-Physical_Facilities: 2.249169435215947 2.249169435215947 0.0', 'Did_Police_Officer_Attend_Scene_of_Accident: -0.9451827242524917 1.0548172757475083 2.0', 'Accident_Severity: 0.9867109634551494 2.9867109634551494 2.0', 'Road_Surface_Conditions: -0.415282392026578 1.584717607973422 2.0', 'Speed_limit: 0.5647840531561457 30.564784053156146 30.0', 'DateVals: -846.4717607973421 1151.528239202658 1998.0', 'Urban_or_Rural_Area: 0.03488372093023262 1.0348837209302326 1.0', 'Weather_Conditions: 0.6395348837209303 1.6395348837209303 1.0', 'Number_of_Casualties: 0.01661129568106312 1.0166112956810631 1.

changes:  48 
  ['Road_Type: -0.023664695843435446 5.583159773876822 5.606824469720258', 'Day_of_Week: 0.009573567365047886 4.109788753347218 4.1002151859821705', 'Junction_Detail: 0.06640884348807452 2.759000297530497 2.6925914540424225', 'Light_Conditions: -0.16204226103787356 1.936328473668551 2.0983707347064247', 'Number_of_Vehicles: 0.06767396547786486 1.5727462064861648 1.5050722410083', '1st_Road_Class: -0.0004766158681892563 3.7643558464742637 3.764832462342453', 'Pedestrian_Crossing-Physical_Facilities: 0.042093751283345915 1.2969354358821779 1.254841684598832', 'Did_Police_Officer_Attend_Scene_of_Accident: 0.004276920862101408 1.1767331151443023 1.172456194282201', 'Accident_Severity: 0.008394420987634543 2.886045819696519 2.8776513987088843', 'Road_Surface_Conditions: -0.0011564634498921755 1.1829812555786967 1.1841377190285889', 'Speed_limit: -0.010099390551811638 30.119012198750372 30.129111589302184', 'DateVals: 93.4014108963845 1693.3445403153823 1599.9431294189978', 'Ur

changes:  46 
  ['Road_Type: 0.0017750395311981748 5.5619116582186825 5.560136618687484', 'Day_of_Week: 0.002376906537844725 4.1194786386676325 4.117101732129788', 'Junction_Detail: -0.012339539670961575 2.7859039343470915 2.798243474018053', 'Light_Conditions: -0.007568296536182073 1.841419261404779 1.8489875579409611', 'Number_of_Vehicles: 0.033744712170544755 1.7200096548394883 1.6862649426689436', '1st_Road_Class: 0.0021578530925134842 3.776249094858798 3.7740912417662846', 'Pedestrian_Crossing-Physical_Facilities: -0.0016958387559364851 1.2261646150132754 1.227860453769212', 'Did_Police_Officer_Attend_Scene_of_Accident: 0.0001228936936574776 1.1930967897658702 1.1929738960722127', 'Accident_Severity: 0.0008908762296804262 2.894279507603186 2.8933886313735058', 'Road_Surface_Conditions: -0.001408654978281021 1.1766835626357712 1.1780922176140523', 'Speed_limit: -0.003994192257202656 30.144822592324402 30.148816784581605', 'DateVals: 1.9447348767298536 1645.7166304610187 1643.771895

changes:  30 
  ['Road_Type: 0.0008947801735965655 5.566144047035767 5.56524926686217', 'Day_of_Week: -0.002404998603888764 4.115874571288584 4.118279569892473', 'Junction_Detail: 0.003130473389359789 2.7809897109260167 2.777859237536657', 'Light_Conditions: -0.00016271995930905447 1.8334149926506615 1.8335777126099706', 'Number_of_Vehicles: -0.00267823403225087 1.7040666340029398 1.7067448680351907', '1st_Road_Class: -0.0015336865038939607 3.7724154826065655 3.7739491691104594', 'Pedestrian_Crossing-Physical_Facilities: 0.0017888419367770858 1.2319941205291525 1.2302052785923754', 'Did_Police_Officer_Attend_Scene_of_Accident: -0.0012490762439396619 1.1888780009799118 1.1901270772238515', 'Accident_Severity: -1.7241849993077096e-05 2.8929446349828516 2.8929618768328447', 'Road_Surface_Conditions: -5.3521576020010286e-05 1.1780989710926018 1.1781524926686218', 'Speed_limit: -0.002084587558183415 30.146986771190594 30.149071358748778', 'DateVals: 1.4865830867988734 1655.0097991180794 165

In [8]:
centrDF

,Road_Type,Day_of_Week,Junction_Detail,Light_Conditions,Number_of_Vehicles,1st_Road_Class,Pedestrian_Crossing-Physical_Facilities,Did_Police_Officer_Attend_Scene_of_Accident,Accident_Severity,Road_Surface_Conditions,Speed_limit,DateVals,Urban_or_Rural_Area,Weather_Conditions,Number_of_Casualties,TimeVals
0,5.566838,4.114790,2.779740,1.830758,1.703458,3.773117,1.232279,1.188864,2.892813,1.178072,30.147167,1655.719892,1.007604,1.215109,1.0,872.007358
1,5.487385,4.109172,2.589248,1.832587,1.703372,3.788729,0.949069,1.195473,2.867012,1.158453,30.188635,531.071681,1.010139,1.157038,1.0,845.481490
2,5.570076,4.019886,2.938447,1.909091,1.952652,3.750000,1.219697,1.139205,2.868371,1.197917,30.217803,1126.644886,1.026515,1.248106,2.0,864.044508


In [10]:
c0 = data.loc[data['clusterID'] == 0]
c1 = data.loc[data['clusterID'] == 1]
c2 = data.loc[data['clusterID'] == 2]

print(c0.shape, c1.shape, c2.shape)

(4077, 29) (4241, 29) (1056, 29)


In [ ]:
#this sections beloow this are simply for my personal understanding and learning/ contribute no value to the project

In [ ]:
for c in testCols:
            if(c in numeric):
                #for numeric sets mean
                new = int(df[c].mean())
                cur = int(centrDF.at[i, c])
                if(new != cur):
                    flag = True
                    centrDF.at[i, c] = new
                    changes += 1
                    val =c + ': ' + str(new - cur) + ' ' + str(new) + ' ' + str(cur)
                    cols1.append(val)
            else:
                #for categorical sets most common value
                new = df[c].value_counts().index.tolist()[0]
                cur = centrDF.at[i, c]
                if( new != cur ):
                    flag = True
                    centrDF.at[i, c] = new
                    changes += 1
                    cols2.append(c + ' ' + str(new) + ' ' + str(cur))

In [ ]:
data[data['clusterID'] ==  1]

In [ ]:
ssetDF = data[data['Accident_Index'].isin(v)]

In [ ]:
testIndxs = ['200501BS70212', '200501BS70289']

lol = data[data['Accident_Index'] == testIndxs[1]]
lol['Latitude'].value_counts().index.tolist()[0]

In [ ]:
data['Accident_Index'].tolist()

In [ ]:
data['TimeVals'].min()

In [ ]:
ssss = data[data['Day_of_Week'].isin([1, 2, 5, 6])]
for i in testCols:
    print(ssss[i].value_counts().index.tolist()[0])

In [ ]:
centrDF['clusterID'].tolist()

In [ ]:
centrDF[centrDF['clusterID'] == 1]

In [ ]:
data.loc[(data['Accident_Index'] == '201001XH30680'), 'Weather_Conditions']

In [ ]:
centrDF.loc[centrDF['clusterID'] == 1]['Weather_Conditions'].tolist()

In [ ]:
centrDF

In [ ]:
centrDF.loc[centrDF['clusterID'] == 1, 'DateVals'] = 400

In [ ]:
centrDF

In [ ]:
data

In [ ]:
two = data[data['clusterID'] == 2]

print(two.shape)
print(two['DateVals'].mean())

In [ ]:
two = data[data['clusterID'] == 1]

print(two.shape)
print(two['DateVals'].mean())

In [ ]:
centrDF

In [ ]:
centrDF.at[2, 'DateVals'] = 81

In [ ]:
centrDF

In [ ]:
centrDF


In [ ]:
data.loc[33].to_dict()